# VMC Implementation
A naive version of VMC is implemented in the directory QauantumMC. A simple test is given here and also in the script `sample_test.py`, where MC is used for the HF wave function of H2 with no Jastrow factor. However, due to limited computation power the the naivety of implementation, my naive MC cannot even precisely reproduce the HF energy, let along optimizing a variational Jastrow factor. Since the only purpose of implementation is to learn VMC, I will not further optimize the implmentation here.

In [1]:
import pyscf
from QuantumMC.VMC.sample import VMC_E
from QuantumMC.ansatz import molecule
import numpy as np

trial_point = np.array([[-0.05317037, -0.07002718,  0.0897823],
                        [-0.00266115,  0.00061476, -0.04018687]])
# [0.1, 0.3, 0.2]])
mol = pyscf.gto.M(atom="H 0 0 0; H 0 0 1", basis="6-31g", spin=0)
mf = pyscf.scf.RHF(mol)
mf.kernel()
mol_SJ = molecule.Pure_det(mf, elec_pos=trial_point, params=np.array([]))
sample, E_mean, E_var, n = VMC_E(1, init_state=trial_point, sigma=0.1, ansatz=mol_SJ,
                                 n_cut=10000, n_interval=20, n_sample=50000, E_only=False,
                                 verbose=True)
print(E_mean, E_var, n)

converged SCF energy = -1.09480796286051
start sampling
samples: 5000/50000
samples: 10000/50000
samples: 15000/50000
samples: 20000/50000
samples: 25000/50000
samples: 30000/50000
samples: 35000/50000
samples: 40000/50000
samples: 45000/50000
-1.0601584314054266 0.1064782285868361 50000


Several improvements can be made to my naive implementation (not made here):
- Calculate the local kinetic energy analytically. It is calculated numeically in my current implementation. We can define the ansatz object in the implementation to record the local information including D, transpose of inverse of D, E_L, v_D, etc. The accessing function may just return the value with a littile computation.
- Update the wave function and local energy after a single-electron move (e.g., [1]). In the current implementation everything is recalculated after perturbation.

Reference
[1] S. Fahy, X. W. Wang, and S. G. Louie, Variational Quantum Monte Carlo Nonlocal Pseudopotential Approach to Solids: Formulation and Application to Diamond, Graphite, and Silicon, Phys. Rev. B 42, 3503 (1990).


# DMC implementation